
#### Create the data frame

In [0]:
from pyspark.sql import types

# Initial data
emp_data_batch1 = [
    (1001, "Alice Johnson",  "IT",        90000),
    (1002, "Bob Smith",      "Finance",   85000),
    (1003, "Charlie Lee",    "Marketing", 78000),
]

emp_schema = types.StructType([
    types.StructField("emp_id", types.IntegerType()),
    types.StructField("emp_name", types.StringType()),
    types.StructField("dept", types.StringType()),
    types.StructField("salary", types.LongType()),
])

emp_df_batch1 = spark.createDataFrame(emp_data_batch1, schema=emp_schema)

display(emp_df_batch1)


#### Create the view

In [0]:
emp_df_batch1.createOrReplaceTempView("emp_view_batch1")

spark.sql("SELECT * FROM emp_view_batch1").display()


#### Create the table from the view

In [0]:
%sql

create or replace table datacentre.bronze.employee
as
select * from emp_view_batch1;

select * from datacentre.bronze.employee;


#### create another data frame for batch 2

In [0]:

emp_data_batch2 = [
    (1001, "Alice Johnson",  "IT",        95000),  # updated salary
    (1002, "Bob Smith",      "Finance",   85000),  # unchanged
    (1004, "Diana King",     "Sales",     88000),  # new
    (1005, "Ethan Brown",    "IT",        99000),  # new
]

emp_df_batch2 = spark.createDataFrame(emp_data_batch2, schema=emp_schema)

display(emp_df_batch2)


#### Create the view from the batch 2 data frame

In [0]:
emp_df_batch2.createOrReplaceTempView("emp_view_batch2")

spark.sql("SELECT * FROM emp_view_batch2").display()

In [0]:
#### Merge and upsert

In [0]:
%sql
merge into datacentre.bronze.employee as target
using emp_view_batch2 as source
on target.emp_id = source.emp_id
when matched then update set
  target.emp_name = source.emp_name,
  target.dept = source.dept,
  target.salary = source.emp_id
when not matched then insert (
  emp_id, emp_name, dept, salary
) values (
  source.emp_id, source.emp_name, source.dept, source.salary
);

select * from datacentre.bronze.employee;

In [0]:
%sql
describe history datacentre.bronze.employee